<a href="https://colab.research.google.com/github/salmanromeo/CS_5783_Machine_Learning_Fall_2022/blob/main/CS_5783_Fall_2022_Assignment_4_Naive_Bayes_Decision_Trees_kNNs/CS_5783_Fall_2022_Assignment_4_Question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 1**

#####See the attached housing data (Asssignment4_Data.xlsx). Each tab in the Excel file contains training and test splits. Your goal is to construct a Naïve Bayes classifier for this data.

1.   Compute and show the conditional probability distribution for each feature. Explain how you got these values and show your work. 
#####Note: You are expected to do this part of the question by hand. Explain how you got the probability distribution for at least two features in detail.
2.   Using your conditional probability table, write a Python code that will compute the probabilities for each example in the test data. Your program should output the probabilities of each class as well as the final classification based on the MAP rule. 
#####Note: You should hard-code the conditional probabilities from the previous step into your code.



**Answer 1**

**Libraries**

In [54]:
import numpy as np
import pandas as pd

**Loading Data**

In [55]:
housing_data = pd.ExcelFile("/content/drive/MyDrive/Asssignment4_Data.xlsx")
train_df = pd.read_excel(housing_data, 'Train', index_col = 'House ID')
test_df = pd.read_excel(housing_data, 'Test', index_col = 'House ID')

In [56]:
train_df

,Local Price,Bathrooms,Land Area,Living area,# Garages,# Rooms,# Bedrooms,Age of home,Construction type
House ID,,,,,,,,,
1,4.9176,1.0,3.4720,0.998,1.0,7,4,42,Apartment
2,5.0208,1.0,3.5310,1.500,2.0,7,4,62,House
3,4.5429,1.0,2.2750,1.175,1.0,6,3,40,Condo
4,4.5573,1.0,4.0500,1.232,1.0,6,3,54,Apartment
5,5.0597,1.0,4.4550,1.121,1.0,6,3,42,Apartment
6,3.8910,1.0,4.4550,0.988,1.0,6,3,56,Condo
7,5.8980,1.0,5.8500,1.240,1.0,7,3,51,Condo
8,5.6039,1.0,9.5200,1.501,0.0,6,3,32,House
9,16.4202,2.5,9.8000,3.420,2.0,10,5,42,Condo


In [57]:
test_df

,Local Price,Bathrooms,Land Area,Living area,# Garages,# Rooms,# Bedrooms,Age of home,Construction type
House ID,,,,,,,,,
24,6.0931,1.5,6.7265,1.652,1.0,6,3,44,Apartment
25,8.3607,1.5,9.1500,1.777,2.0,8,4,48,House
26,8.1400,1.0,8.0000,1.504,2.0,7,3,3,House
27,9.1416,1.5,7.3262,1.831,1.5,8,4,31,Apartment
28,12.0000,1.5,5.0000,1.200,2.0,6,3,30,Apartment


**1.1 By Hand Results**

**Mean and Standard Deviation (SD) Table**

In [58]:
Construction_Type = ['Apartment', 'Condo', 'House']  
# Mean and Standard Deviation (SD) for each feature 
Local_Price_Mean   = [7.3327, 7.4159, 5.7607] 
Local_Price_SD     = [3.6160, 4.6112, 0.5701] 
Bathrooms_Mean     = [1.2857, 1.3333, 1.0714]
Bathrooms_SD       = [0.5669, 0.6055, 0.1890]
Land_Area_Mean     = [6.1039, 6.0250, 6.6309]
Land_Area_SD       = [3.2585, 2.5448, 2.2490]
Living_Area_Mean   = [1.5050, 1.5533, 1.3917]
Living_Area_SD     = [0.7041, 0.9234, 0.2129]
Garages_Mean       = [1.2143, 1.3333, 1.0714]
Garages_SD         = [0.6986, 0.5164, 0.8381]
Rooms_Mean         = [6.8571, 6.8333, 6.1429]
Rooms_SD           = [1.3452, 1.6021, 0.6901]
Bedrooms_Mean      = [3.4286, 3.3333, 3.0000]
Bedrooms_SD        = [0.9759, 0.8165, 0.5774]
Age_of_Home_Mean   = [38.7143, 39.6667, 34.2857]
Age_of_Home_SD     = [14.6824, 13.9523, 12.7242]

# Create table for each feature Mean and Standard Deviation 
list_of_tuples = list(zip(Construction_Type, Local_Price_Mean, Local_Price_SD, Bathrooms_Mean, Bathrooms_SD, Land_Area_Mean, Land_Area_SD, Living_Area_Mean, Living_Area_SD, Garages_Mean, Garages_SD, Rooms_Mean, Rooms_SD, Bedrooms_Mean, Bedrooms_SD, Age_of_Home_Mean, Age_of_Home_SD))   
Mean_SD_df = pd.DataFrame(list_of_tuples, columns = ['Construction Type', 'Local Price Mean','Local Price SD','Bathrooms Mean','Bathrooms SD','Land Area Mean','Land Area SD','Living Area Mean','Living Area SD','Garages Mean','Garages SD','Rooms Mean','Rooms SD','Bedrooms Mean','Bedrooms SD','Age of Home Mean','Age of Home SD'])  

In [59]:
Mean_SD_df

,Construction Type,Local Price Mean,Local Price SD,Bathrooms Mean,Bathrooms SD,Land Area Mean,Land Area SD,Living Area Mean,Living Area SD,Garages Mean,Garages SD,Rooms Mean,Rooms SD,Bedrooms Mean,Bedrooms SD,Age of Home Mean,Age of Home SD
0,Apartment,7.3327,3.6160,1.2857,0.5669,6.1039,3.2585,1.5050,0.7041,1.2143,0.6986,6.8571,1.3452,3.4286,0.9759,38.7143,14.6824
1,Condo,7.4159,4.6112,1.3333,0.6055,6.0250,2.5448,1.5533,0.9234,1.3333,0.5164,6.8333,1.6021,3.3333,0.8165,39.6667,13.9523
2,House,5.7607,0.5701,1.0714,0.1890,6.6309,2.2490,1.3917,0.2129,1.0714,0.8381,6.1429,0.6901,3.0000,0.5774,34.2857,12.7242


**Conditional Probability Table**

**House ID 24**

In [60]:
# House ID 24
Construction_Type = ['Apartment', 'Condo', 'House']  
# Conditional Probability for each feature 
Local_Price_Probability   = [0.1040, 0.0830, 0.5904] 
Bathrooms_Probability     = [0.6552, 0.6344, 0.1613]
Land_Area_Probability     = [0.1202, 0.1509, 0.1772]
Living_Area_Probability   = [0.5544, 0.4296, 0.8874]
Garages_Probability       = [0.5448, 0.6273, 0.4742]
Rooms_Probability         = [0.2421, 0.2175, 0.5658]
Bedrooms_Probability      = [0.3712, 0.4495, 0.6909]
Age_of_Home_Probability   = [0.0255, 0.0272, 0.0234]

# Create table of Conditional Probability for House ID 24
list_of_tuples = list(zip(Construction_Type, Local_Price_Probability, Bathrooms_Probability, Land_Area_Probability, Living_Area_Probability, Garages_Probability, Rooms_Probability, Bedrooms_Probability, Age_of_Home_Probability))   
Probability24_df = pd.DataFrame(list_of_tuples, columns = ['Construction Type', 'Local Price Probability','Bathrooms Probability','Land Area Probability','Living Area Probability','Garages Probability','Rooms Probability','Bedrooms Probability','Age_of_Home_Probability'])  

In [61]:
Probability24_df

,Construction Type,Local Price Probability,Bathrooms Probability,Land Area Probability,Living Area Probability,Garages Probability,Rooms Probability,Bedrooms Probability,Age_of_Home_Probability
0,Apartment,0.1040,0.6552,0.1202,0.5544,0.5448,0.2421,0.3712,0.0255
1,Condo,0.0830,0.6344,0.1509,0.4296,0.6273,0.2175,0.4495,0.0272
2,House,0.5904,0.1613,0.1772,0.8874,0.4742,0.5658,0.6909,0.0234


**House ID 25**

In [62]:
# House ID 25
Construction_Type = ['Apartment', 'Condo', 'House']  
# Conditional Probability for each feature 
Local_Price_Probability   = [0.1060, 0.0847, 0.00002131] 
Bathrooms_Probability     = [0.6552, 0.6344, 0.1613]
Land_Area_Probability     = [0.0791, 0.0738, 0.0947]
Living_Area_Probability   = [0.5259, 0.4195, 0.3643]
Garages_Probability       = [0.3034, 0.3357, 0.2577]
Rooms_Probability         = [0.2067, 0.1910, 0.0155]
Bedrooms_Probability      = [0.3444, 0.3501, 0.1542]
Age_of_Home_Probability   = [0.0222, 0.0239, 0.0175]

# Create table of Conditional Probability for House ID 25 
list_of_tuples = list(zip(Construction_Type, Local_Price_Probability, Bathrooms_Probability, Land_Area_Probability, Living_Area_Probability, Garages_Probability, Rooms_Probability, Bedrooms_Probability, Age_of_Home_Probability))   
Probability25_df = pd.DataFrame(list_of_tuples, columns = ['Construction Type', 'Local Price Probability','Bathrooms Probability','Land Area Probability','Living Area Probability','Garages Probability','Rooms Probability','Bedrooms Probability','Age_of_Home_Probability'])  

In [63]:
Probability25_df

,Construction Type,Local Price Probability,Bathrooms Probability,Land Area Probability,Living Area Probability,Garages Probability,Rooms Probability,Bedrooms Probability,Age_of_Home_Probability
0,Apartment,0.106000,0.6552,0.0791,0.5259,0.3034,0.2067,0.3444,0.0222
1,Condo,0.084700,0.6344,0.0738,0.4195,0.3357,0.1910,0.3501,0.0239
2,House,0.000021,0.1613,0.0947,0.3643,0.2577,0.0155,0.1542,0.0175


**House ID 26**

In [64]:
# House ID 26
Construction_Type = ['Apartment', 'Condo', 'House']  
# Conditional Probability for each feature 
Local_Price_Probability   = [0.1076, 0.0855, 0.00001155] 
Bathrooms_Probability     = [0.6198, 0.5662, 1.9654]
Land_Area_Probability     = [0.1034, 0.1160, 0.1474]
Living_Area_Probability   = [0.5666, 0.4314, 1.6305]
Garages_Probability       = [0.3034, 0.3357, 0.2577]
Rooms_Probability         = [0.2949, 0.2477, 0.2673]
Bedrooms_Probability      = [0.3712, 0.4495, 0.6909]
Age_of_Home_Probability   = [0.00141, 0.00009048, 0.001526]

# Create table of Conditional Probability for House ID 26 
list_of_tuples = list(zip(Construction_Type, Local_Price_Probability, Bathrooms_Probability, Land_Area_Probability, Living_Area_Probability, Garages_Probability, Rooms_Probability, Bedrooms_Probability, Age_of_Home_Probability))   
Probability26_df = pd.DataFrame(list_of_tuples, columns = ['Construction Type', 'Local Price Probability','Bathrooms Probability','Land Area Probability','Living Area Probability','Garages Probability','Rooms Probability','Bedrooms Probability','Age_of_Home_Probability'])  

In [65]:
Probability26_df

,Construction Type,Local Price Probability,Bathrooms Probability,Land Area Probability,Living Area Probability,Garages Probability,Rooms Probability,Bedrooms Probability,Age_of_Home_Probability
0,Apartment,0.107600,0.6198,0.1034,0.5666,0.3034,0.2949,0.3712,0.001410
1,Condo,0.085500,0.5662,0.1160,0.4314,0.3357,0.2477,0.4495,0.000090
2,House,0.000012,1.9654,0.1474,1.6305,0.2577,0.2673,0.6909,0.001526


**House ID 27**

In [66]:
# House ID 27
Construction_Type = ['Apartment', 'Condo', 'House']  
# Conditional Probability for each feature 
Local_Price_Probability   = [0.0973, 0.0807, 0.00000001615] 
Bathrooms_Probability     = [0.6552, 0.6344, 0.1613]
Land_Area_Probability     = [0.1141, 0.1376, 0.1691]
Living_Area_Probability   = [0.5090, 0.4129, 0.2229]
Garages_Probability       = [0.5252, 0.7333, 0.4177]
Rooms_Probability         = [0.2067, 0.1910, 0.0155]
Bedrooms_Probability      = [0.3444, 0.3501, 0.1542]
Age_of_Home_Probability   = [0.0237, 0.0236, 0.0303]

# Create table of Conditional Probability for House ID 27
list_of_tuples = list(zip(Construction_Type, Local_Price_Probability, Bathrooms_Probability, Land_Area_Probability, Living_Area_Probability, Garages_Probability, Rooms_Probability, Bedrooms_Probability, Age_of_Home_Probability))   
Probability27_df = pd.DataFrame(list_of_tuples, columns = ['Construction Type', 'Local Price Probability','Bathrooms Probability','Land Area Probability','Living Area Probability','Garages Probability','Rooms Probability','Bedrooms Probability','Age_of_Home_Probability'])  

In [67]:
Probability27_df

,Construction Type,Local Price Probability,Bathrooms Probability,Land Area Probability,Living Area Probability,Garages Probability,Rooms Probability,Bedrooms Probability,Age_of_Home_Probability
0,Apartment,9.730000e-02,0.6552,0.1141,0.5090,0.5252,0.2067,0.3444,0.0237
1,Condo,8.070000e-02,0.6344,0.1376,0.4129,0.7333,0.1910,0.3501,0.0236
2,House,1.615000e-08,0.1613,0.1691,0.2229,0.4177,0.0155,0.1542,0.0303


**House ID 28**

In [68]:
# House ID 28
Construction_Type = ['Apartment', 'Condo', 'House']  
# Conditional Probability for each feature 
Local_Price_Probability   = [0.0480, 0.0528, 0.000000000000000000000000006854] 
Bathrooms_Probability     = [0.6552, 0.6344, 0.1613]
Land_Area_Probability     = [0.1156, 0.1446, 0.1364]
Living_Area_Probability   = [0.5159, 0.4015, 1.2493]
Garages_Probability       = [0.3034, 0.3357, 0.2577]
Rooms_Probability         = [0.2421, 0.2175, 0.5658]
Bedrooms_Probability      = [0.3712, 0.4495, 0.6909]
Age_of_Home_Probability   = [0.0228, 0.0225, 0.0296]

# Create table of Conditional Probability for House ID 28 
list_of_tuples = list(zip(Construction_Type, Local_Price_Probability, Bathrooms_Probability, Land_Area_Probability, Living_Area_Probability, Garages_Probability, Rooms_Probability, Bedrooms_Probability, Age_of_Home_Probability))   
Probability28_df = pd.DataFrame(list_of_tuples, columns = ['Construction Type', 'Local Price Probability','Bathrooms Probability','Land Area Probability','Living Area Probability','Garages Probability','Rooms Probability','Bedrooms Probability','Age_of_Home_Probability'])  

In [69]:
Probability28_df

,Construction Type,Local Price Probability,Bathrooms Probability,Land Area Probability,Living Area Probability,Garages Probability,Rooms Probability,Bedrooms Probability,Age_of_Home_Probability
0,Apartment,4.800000e-02,0.6552,0.1156,0.5159,0.3034,0.2421,0.3712,0.0228
1,Condo,5.280000e-02,0.6344,0.1446,0.4015,0.3357,0.2175,0.4495,0.0225
2,House,6.854000e-27,0.1613,0.1364,1.2493,0.2577,0.5658,0.6909,0.0296


**1.2 MAP Rule**

**House ID 24**

In [70]:
# House ID 24

# Number of Apartment = 7
# Number of Condo = 6
# Number of House = 7
# Total Number of Sample = 20

Apartment_Probability = (0.1040 * 0.6552 * 0.1202 * 0.5544 * 0.5448 * 0.2421 * 0.3712 * 0.0255) * (7/20)
print(Apartment_Probability)

Condo_Probability = (0.083 * 0.6344 * 0.1509 * 0.4296 * 0.6273 * 0.2175 * 0.4495 * 0.0272) * (6/20)
print(Condo_Probability)

House_Probability = (0.5904 * 0.1613 * 0.1772 * 0.8874 * 0.4742 * 0.5658 * 0.6909 * 0.0234) * (7/20)
print(House_Probability)

# MAP rule selects the maximum of these probabilities
map_classify = max(Apartment_Probability, Condo_Probability, House_Probability)
if (map_classify==Apartment_Probability):
  print("House#24 is classified as APARTMENT")
if (map_classify==Condo_Probability):
  print("House#24 is classified as CONDO")
if (map_classify==House_Probability):
  print("House#24 is classified as HOUSE")

1.9841888188414975e-06
1.7082411633024023e-06
2.273460669294739e-05
House#24 is classified as HOUSE


**House ID 25**

In [71]:
# House ID 25

Apartment_Probability = (0.1060 *	0.6552 * 0.0791 * 0.5259 * 0.3034 * 0.2067 * 0.3444 * 0.0222) * (7/20)
print(Apartment_Probability)

Condo_Probability = (0.0847 *	0.6344 * 0.0738 * 0.4195 * 0.3357 * 0.1910 * 0.3501 * 0.0239) * (6/20)
print(Condo_Probability)

House_Probability = (0.00002131 *	0.1613 * 0.0947 * 0.3643 * 0.2577 * 0.0155 * 0.1542 * 0.0175) * (7/20)
print(House_Probability)

# MAP rule selects the maximum of these probabilities
map_classify = max(Apartment_Probability, Condo_Probability, House_Probability)
if (map_classify==Apartment_Probability):
  print("House#25 is classified as APARTMENT")
if (map_classify==Condo_Probability):
  print("House#25 is classified as CONDO")
if (map_classify==House_Probability):
  print("House#25 is classified as HOUSE")

4.84841317817334e-07
2.677508674735345e-07
4.473665937135081e-13
House#25 is classified as APARTMENT


**House ID 26**

In [72]:
# House ID 26

Apartment_Probability = (0.1076 * 0.6198 * 0.1034 * 0.5666 * 0.3034 * 0.2949 * 0.3712 * 0.001410) * (7/20)
print(Apartment_Probability)

Condo_Probability = (0.0855 *	0.5662 * 0.1160 * 0.4314 * 0.3357 * 0.2477 * 0.4495 * 0.00009048) * (6/20)
print(Condo_Probability)

House_Probability = (0.00001155 * 1.9654 * 0.1474 * 1.6305 * 0.2577 * 0.2673 * 0.6909 * 0.001526) * (7/20)
print(House_Probability)

# MAP rule selects the maximum of these probabilities
map_classify = max(Apartment_Probability, Condo_Probability, House_Probability)
if (map_classify==Apartment_Probability):
  print("House#26 is classified as APARTMENT")
if (map_classify==Condo_Probability):
  print("House#26 is classified as CONDO")
if (map_classify==House_Probability):
  print("House#26 is classified as HOUSE")

6.403927782832104e-08
2.4578478803659445e-09
1.386763398413728e-10
House#26 is classified as APARTMENT


**House ID 27**

In [73]:
# House ID 27

Apartment_Probability = (0.0973 *	0.6552 * 0.1141 * 0.5090 * 0.5252 * 0.2067 * 0.3444 * 0.0237) * (7/20)
print(Apartment_Probability)

Condo_Probability = (0.0807 *	0.6344 * 0.1376 * 0.4129 * 0.7333 * 0.1910 * 0.3501 * 0.0236) * (6/20)
print(Condo_Probability)

House_Probability = (1.615e-08 * 0.1613 * 0.1691 * 0.2229 * 0.4177 * 0.0155 * 0.1542 * 0.0303) * (7/20)
print(House_Probability)

# MAP rule selects the maximum of these probabilities
map_classify = max(Apartment_Probability, Condo_Probability, House_Probability)
if (map_classify==Apartment_Probability):
  print("House#27 is classified as APARTMENT")
if (map_classify==Condo_Probability):
  print("House#27 is classified as CONDO")
if (map_classify==House_Probability):
  print("House#27 is classified as HOUSE")

1.1482458460353102e-06
1.009811812169203e-06
1.039565337168584e-15
House#27 is classified as APARTMENT


**House ID 28**

In [74]:
# House ID 28

Apartment_Probability = (0.0480 *	0.6552 * 0.1156 * 0.5159 * 0.3034 * 0.2421 * 0.3712 * 0.0228) * (7/20)
print(Apartment_Probability)

Condo_Probability = (0.0528 *	0.6344 * 0.1446 * 0.4015 * 0.3357 * 0.2175 * 0.4495 * 0.0225) * (6/20)
print(Condo_Probability)

House_Probability = (6.854e-27 * 0.1613 * 0.1364 * 1.2493 * 0.2577 * 0.5658 * 0.6909 * 0.0296) * (7/20)
print(House_Probability)

# MAP rule selects the maximum of these probabilities
map_classify = max(Apartment_Probability, Condo_Probability, House_Probability)
if (map_classify==Apartment_Probability):
  print("House#28 is classified as APARTMENT")
if (map_classify==Condo_Probability):
  print("House#28 is classified as CONDO")
if (map_classify==House_Probability):
  print("House#28 is classified as HOUSE")

4.080935332355422e-07
4.308191612306969e-07
1.9661284501693058e-31
House#28 is classified as CONDO
